In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [4]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor1 = torch.tensor([word_to_ix["world"]], dtype=torch.long)
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
hello_embed1 = embeds(lookup_tensor1)
print(hello_embed)
print(hello_embed1)

tensor([ 0])
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]])
tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]])


In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 5
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        print(embeds)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

print(len(vocab))
for epoch in range(1):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        #print(context_idxs, target)
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
97
tensor([[-0.3978, -1.9291,  0.9264,  0.9134, -0.7902, -1.2617,  1.9147,
         -1.8613,  0.4739,  0.9988]])
tensor([[-1.2617,  1.9147, -1.8613,  0.4739,  0.9988,  0.4841, -0.7030,
         -0.8268,  0.1012,  0.1952]])
tensor([[ 0.4842, -0.7030, -0.8269,  0.1013,  0.1953, -0.9773,  0.8748,
          0.9873,  0.2505, -0.7930]])
tensor([[-0.9773,  0.8749,  0.9872,  0.2504, -0.7930, -0.8044, -1.0371,
         -1.0669, -0.2085, -0.2155]])
tensor([[-0.8044, -1.0372, -1.0669, -0.2085, -0.2157,  1.9907, -0.9247,
         -0.9301,  1.4301,  0.4208]])
tensor([[ 1.9907, -0.9248, -0.9301,  1.4301,  0.4208, -1.3034,  0.5190,
         -0.5464, -0.8024,  0.0186]])
tensor([[-1.3033,  0.5190, -0.5463, -0.8023,  0.0185, -0.5458,  1.1528,
         -0.3997,  0.9738,  1.5453]])
tensor([[-0.5458,  1.1529, -0.3997,  0.9738,  1.5453,  1.2563,  0.5000,
          0.0402,  0.4647, -3.3793]])
tensor([[ 1.2563

In [6]:
sent = "this is a test and it's working perfect. it would really nice and is working".split()

In [25]:
sent;

In [23]:
voc = set(sent)
voc;

In [21]:
word_to_idx = {word:i for word, i in enumerate(voc)}

In [27]:
word_to_idx

{0: 'it',
 1: 'would',
 2: 'this',
 3: 'working',
 4: "it's",
 5: 'and',
 6: 'is',
 7: 'perfect.',
 8: 'test',
 9: 'really',
 10: 'a',
 11: 'nice'}